In [28]:
import os
from datetime import datetime
import sqlite3
import pandas as pd
import numpy as np

from pathlib import Path
from copy import copy
from typing import Union, Optional

import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.formatting.rule import Rule
from openpyxl.styles.differential import DifferentialStyle

In [29]:
def copy_excel_cell_range(
        src_ws: openpyxl.worksheet.worksheet.Worksheet,
        min_row: int = None,
        max_row: int = None,
        min_col: int = None,
        max_col: int = None,
        tgt_ws: openpyxl.worksheet.worksheet.Worksheet = None,
        tgt_min_row: int = 1,
        tgt_min_col: int = 1,
        with_style: bool = True
) -> openpyxl.worksheet.worksheet.Worksheet:
    """
    copies all cells from the source worksheet [src_ws] starting from [min_row] row
    and [min_col] column up to [max_row] row and [max_col] column
    to target worksheet [tgt_ws] starting from [tgt_min_row] row
    and [tgt_min_col] column.

    @param src_ws:  source worksheet
    @param min_row: smallest row index in the source worksheet (1-based index)
    @param max_row: largest row index in the source worksheet (1-based index)
    @param min_col: smallest column index in the source worksheet (1-based index)
    @param max_col: largest column index in the source worksheet (1-based index)
    @param tgt_ws:  target worksheet.
                    If None, then the copy will be done to the same (source) worksheet.
    @param tgt_min_row: target row index (1-based index)
    @param tgt_min_col: target column index (1-based index)
    @param with_style:  whether to copy cell style. Default: True

    @return: target worksheet object
    """
    if tgt_ws is None:
        tgt_ws = src_ws

    # https://stackoverflow.com/a/34838233/5741205
    for row in src_ws.iter_rows(min_row=min_row, max_row=max_row,
                                min_col=min_col, max_col=max_col):
        for cell in row:
            tgt_cell = tgt_ws.cell(
                row=cell.row + tgt_min_row - 1,
                column=cell.col_idx + tgt_min_col - 1,
                value=cell.value
            )
            if with_style and cell.has_style:
                # tgt_cell._style = copy(cell._style)
                tgt_cell.font = copy(cell.font)
                tgt_cell.border = copy(cell.border)
                tgt_cell.fill = copy(cell.fill)
                tgt_cell.number_format = copy(cell.number_format)
                tgt_cell.protection = copy(cell.protection)
                tgt_cell.alignment = copy(cell.alignment)
    return tgt_ws

In [30]:
def append_df_to_excel(
        filename: Union[str, Path],
        df: pd.DataFrame,
        sheet_name: str = 'Sheet1',
        startrow: Optional[int] = None,
        max_col_width: int = 30,
        autofilter: bool = False,
        fmt_int: str = "#,##0",
        fmt_float: str = "#,##0.00",
        fmt_date: str = "yyyy-mm-dd",
        fmt_datetime: str = "yyyy-mm-dd hh:mm",
        truncate_sheet: bool = False,
        storage_options: Optional[dict] = None,
        **to_excel_kwargs
) -> None:
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    @param filename: File path or existing ExcelWriter
                     (Example: '/path/to/file.xlsx')
    @param df: DataFrame to save to workbook
    @param sheet_name: Name of sheet which will contain DataFrame.
                       (default: 'Sheet1')
    @param startrow: upper left cell row to dump data frame.
                     Per default (startrow=None) calculate the last row
                     in the existing DF and write to the next row...
    @param max_col_width: maximum column width in Excel. Default: 40
    @param autofilter: boolean - whether add Excel autofilter or not. Default: False
    @param fmt_int: Excel format for integer numbers
    @param fmt_float: Excel format for float numbers
    @param fmt_date: Excel format for dates
    @param fmt_datetime: Excel format for datetime's
    @param truncate_sheet: truncate (remove and recreate) [sheet_name]
                           before writing DataFrame to Excel file
    @param storage_options: dict, optional
        Extra options that make sense for a particular storage connection, e.g. host, port,
        username, password, etc., if using a URL that will be parsed by fsspec, e.g.,
        starting “s3://”, “gcs://”.
    @param to_excel_kwargs: arguments which will be passed to `DataFrame.to_excel()`
                            [can be a dictionary]
    @return: None

    Usage examples:

    >>> append_df_to_excel('/tmp/test.xlsx', df, autofilter=True,
                           freeze_panes=(1,0))

    >>> append_df_to_excel('/tmp/test.xlsx', df, header=None, index=False)

    >>> append_df_to_excel('/tmp/test.xlsx', df, sheet_name='Sheet2',
                           index=False)

    >>> append_df_to_excel('/tmp/test.xlsx', df, sheet_name='Sheet2',
                           index=False, startrow=25)

    >>> append_df_to_excel('/tmp/test.xlsx', df, index=False,
                           fmt_datetime="dd.mm.yyyy hh:mm")

    (c) [MaxU](https://stackoverflow.com/users/5741205/maxu?tab=profile)
    """
    def set_column_format(ws, column_letter, fmt):
        for cell in ws[column_letter]:
            cell.number_format = fmt
    filename = Path(filename)
    file_exists = filename.is_file()
    # process parameters
    # calculate first column number
    # if the DF will be written using `index=True`, then `first_col = 2`, else `first_col = 1`
    first_col = int(to_excel_kwargs.get("index", True)) + 1
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')
    # save content of existing sheets
    if file_exists:
        wb = load_workbook(filename)
        sheet_names = wb.sheetnames
        sheet_exists = sheet_name in sheet_names
        sheets = {ws.title: ws for ws in wb.worksheets}

    with pd.ExcelWriter(
        filename.with_suffix(".xlsx"),
        engine="openpyxl",
        mode="a" if file_exists else "w",
        if_sheet_exists="new" if file_exists else None,
        date_format=fmt_date,
        datetime_format=fmt_datetime,
        storage_options=storage_options
    ) as writer:
        if file_exists:
            # try to open an existing workbook
            writer.book = wb
            # get the last row in the existing Excel sheet
            # if it was not specified explicitly
            if startrow is None and sheet_name in writer.book.sheetnames:
                startrow = writer.book[sheet_name].max_row
            # truncate sheet
            if truncate_sheet and sheet_name in writer.book.sheetnames:
                # index of [sheet_name] sheet
                idx = writer.book.sheetnames.index(sheet_name)
                # remove [sheet_name]
                writer.book.remove(writer.book.worksheets[idx])
                # create an empty sheet [sheet_name] using old index
                writer.book.create_sheet(sheet_name, idx)
            # copy existing sheets
            writer.sheets = sheets
        else:
            # file doesn't exist, we are creating a new one
            startrow = 0

        # write out the DataFrame to an ExcelWriter
        df.to_excel(writer, sheet_name=sheet_name, **to_excel_kwargs)
        worksheet = writer.sheets[sheet_name]

        if autofilter:
            worksheet.auto_filter.ref = worksheet.dimensions

        for xl_col_no, dtyp in enumerate(df.dtypes, first_col):
            col_no = xl_col_no - first_col
            width = max(df.iloc[:, col_no].astype(str).str.len().max(),
                        len(df.columns[col_no]) + 6)
            width = min(max_col_width, width)
            column_letter = get_column_letter(xl_col_no)
            worksheet.column_dimensions[column_letter].width = width
            if np.issubdtype(dtyp, np.integer):
                set_column_format(worksheet, column_letter, fmt_int)
            if np.issubdtype(dtyp, np.floating):
                set_column_format(worksheet, column_letter, fmt_float)

    if file_exists and sheet_exists:
        # move (append) rows from new worksheet to the `sheet_name` worksheet
        wb = load_workbook(filename)
        # retrieve generated worksheet name
        new_sheet_name = set(wb.sheetnames) - set(sheet_names)
        if new_sheet_name:
            new_sheet_name = list(new_sheet_name)[0]
        # copy rows written by `df.to_excel(...)` to
        copy_excel_cell_range(
            src_ws=wb[new_sheet_name],
            tgt_ws=wb[sheet_name],
            tgt_min_row=startrow + 1,
            with_style=True
        )
        # remove new (generated by Pandas) worksheet
        del wb[new_sheet_name]
        wb.save(filename)
        wb.close()

In [31]:
def insert_data_into_db(db_name, cur):
    # Replaces any spaces, &, - characters into underscore
    TABLENAME_CLEANING_CHARACTERS = [(" ", "_"),("&", "_"),("-", "_")]
    
       
    before_file = open(COMPARISON_FILE_PATH + "/" + db_name + BEFORE_IMPLMENT_KEYWORD + COMPARISON_FILE_EXTENSION, "r")
    after_file = open(COMPARISON_FILE_PATH + "/" + db_name + AFTER_IMPLMENT_KEYWORD + COMPARISON_FILE_EXTENSION, "r")
    before_db_data = before_file.read().split("\n\n")
    after_db_data = after_file.read().split("\n\n")
    
    
    for index, table in enumerate(before_db_data + after_db_data):
        
        if index < len(before_db_data):
            table_suffix = BEFORE_IMPLMENT_KEYWORD
        else:
            table_suffix = AFTER_IMPLMENT_KEYWORD

        # Get the data of individual tables
        table_info = table.split("\n")
        
        if table_info != '' and len(table_info) > 1:
            
            # Cleaning and storing the table name
            table_name = table_info[0][len("*** ") : -len(" ***")]
            table_name = table_name + "_" + table_suffix
            for clean_char in TABLENAME_CLEANING_CHARACTERS:
                table_name = table_name.replace(clean_char[0], clean_char[1])
            
            # Cleaning and storing the table columns
            table_columns = list()
            for column in table_info[1].split("\t"):
                table_columns.append(column.replace(".", "_"))
            table_columns = tuple(table_columns)
            
            # Creating tables and execution
            table_creation_query = 'CREATE TABLE {table_name} {table_columns}'
            table_creation_query = table_creation_query.format(table_name = table_name, table_columns = table_columns)
            try:
                cur.execute(table_creation_query)
            except sqlite3.OperationalError:
                pass
            
            for data in table_info[2:]:
                
                # Creating table data and execution
                row_data = tuple(data.split("\t"))
                table_data_insertion_query = 'INSERT INTO {table_name} VALUES {table_data}'
                table_data_insertion_query = table_data_insertion_query.format(table_name = table_name, table_data = row_data)
                cur.execute(table_data_insertion_query)
            
            con.commit()
            
    before_file.close()
    after_file.close()

In [32]:
def create_result_df(con, table_name):
    before_table = table_name + "_" + BEFORE_IMPLMENT_KEYWORD
    after_table = table_name + "_" + AFTER_IMPLMENT_KEYWORD
    
    # Get all the columns for the current table
    get_columns_query = "SELECT * FROM {table_name};".format(table_name = before_table)
    data = con.cursor().execute(get_columns_query)
    columns = [col[0] for col in data.description]
    
    # Custom and formatted columns query
    display_columns_1 = "b." + columns[0] + " AS before_date, a." + columns[0] + " AS after_date, "
    display_columns_2 = "a." + columns[0] + " AS before_date, b." + columns[0] + " AS after_Date, "
    for column in columns[1:-1]:
        display_columns_1 += "a." + column + ", "
        display_columns_2 += "a." + column + ", "
    display_columns_2 += "a." + columns[-1] + " AS count_before, b." + columns[-1] + " AS count_after"
    display_columns_1 += "b." + columns[-1] + " AS count_before, a." + columns[-1] + " AS count_after"
    
    # Join Condition query
    join_condition = ""
    for column in columns[1:-2]:
        join_condition += "a." + column + " = b." + column + " AND "
    join_condition += "a." + columns[-2] + " = b." + columns[-2]
    
    # Combine the queries: display + join
    query1 = "SELECT {display_columns} from {after_table} AS a LEFT JOIN {before_table} AS b ON {join_condition}"
    query1 = query1.format(display_columns = display_columns_1, after_table = after_table, before_table = before_table, join_condition = join_condition)
    query2 = "SELECT {display_columns} from {before_table} AS a LEFT JOIN {after_table} AS b ON {join_condition}"
    query2 = query2.format(display_columns = display_columns_2, before_table = before_table, after_table = after_table, join_condition = join_condition)
    query = "{} UNION {}".format(query1, query2)
    
    # Data Cleaning
    new_df = pd.read_sql(query, con)
    new_df[["count_before","count_after"]] = new_df[["count_before","count_after"]].fillna(0)
    new_df[["count_before","count_after"]] = new_df[["count_before","count_after"]].applymap(int)
    new_df["Difference (after - before)"] = new_df["count_after"] - new_df["count_before"]
    # Sorting by the join condition columns
    new_df = new_df.sort_values(new_df.columns[2:-3].tolist())
    
    return new_df
    

In [33]:
dxf2 = DifferentialStyle(fill = PatternFill(bgColor = "FFC7CE"))# Red Fill
dxf = DifferentialStyle(fill = PatternFill(bgColor = "00CCFFCC"))# Green Fill
g = Rule(type = "expression", dxf = dxf2, stopIfTrue = True)
r = Rule(type = "expression", dxf = dxf, stopIfTrue = True)
merged_cell_font = Font(bold = True, color = "00FFFFFF")
merged_cell_fill = PatternFill("solid", fgColor = "00003366")
merged_cell_alignment = Alignment(horizontal = "center", vertical = "center")

def write_to_excel_with_formatting(df, sheet_name):
    # Write and style the table name
    wb = load_workbook(RESULT_FILENAME)
    ws = wb[sheet_name]
    current_row = ws.max_row + 2
    table_name_cell = "B" + str(current_row)
    ws[table_name_cell] = sheet_name
    ws.merge_cells(table_name_cell + ":" + get_column_letter(df.shape[1] + 1) + str(current_row))
    merged_cell = ws[table_name_cell]
    merged_cell.font = merged_cell_font
    merged_cell.fill = merged_cell_fill
    merged_cell.alignment = merged_cell_alignment
    wb.save(RESULT_FILENAME)
    
    append_df_to_excel(filename = RESULT_FILENAME, df = df, sheet_name = sheet_name, startrow = current_row, index = True)
    
    # Add styling to highlight the difference
    wb = load_workbook(RESULT_FILENAME)
    ws = wb[sheet_name]
    current_row += 1
    max_row = ws.max_row
    g.formula = ['ISBLANK($C' + str(current_row) + ')']
    r.formula = ['ISBLANK($B' + str(current_row) + ')']
    ws.conditional_formatting.add("B" + str(current_row) + ":" + get_column_letter(df.shape[1] + 1) + str(max_row), r)
    ws.conditional_formatting.add("B" + str(current_row) + ":" + get_column_letter(df.shape[1] + 1) + str(max_row), g)
    for index in range(current_row, max_row + 1):
        ws[get_column_letter(df.shape[1] + 1) + str(index)].number_format = '#,##0'
    ws.column_dimensions['A'].hidden = True
    for column_cells in ws.columns: 
        unmerged_cells = list(filter(lambda cell_to_check: cell_to_check.coordinate not in ws.merged_cells, column_cells)) 
        length = max(len(str(cell.value)) for cell in unmerged_cells) 
        ws.column_dimensions[unmerged_cells[0].column_letter].width = length * 1.2
    wb.save(RESULT_FILENAME)

In [34]:
COMPARISON_FILE_PATH = "./files"
COMPARISON_FILE_EXTENSION = ".txt"
BEFORE_IMPLMENT_KEYWORD = "Before"
AFTER_IMPLMENT_KEYWORD = "After"

In [35]:
before_data = set()
after_data = set()
comparison_data = list()

with os.scandir(COMPARISON_FILE_PATH) as current_directory:
    for entry in current_directory:
        
        # Criteria for selecting the required database
        criteria_set = set()
        criteria_1 = entry.name.endswith(COMPARISON_FILE_EXTENSION)
        criteria_2 = entry.name.find(BEFORE_IMPLMENT_KEYWORD) > -1
        criteria_3 = entry.name.find(AFTER_IMPLMENT_KEYWORD) > -1
        criteria_set.add(criteria_1 and (criteria_2 or criteria_3))
        
        # Add names of the database that satisfy the criteria
        if False not in criteria_set:
            if criteria_2:
                before_data.add(entry.name[ : -len(BEFORE_IMPLMENT_KEYWORD + COMPARISON_FILE_EXTENSION)])
            else:
                after_data.add(entry.name[ : -len(AFTER_IMPLMENT_KEYWORD + COMPARISON_FILE_EXTENSION)])

# Filter out files that only has one file
common_data = set.intersection(before_data, after_data)

# Warn the user that if any files were dropped
check_1 = set.difference(before_data, common_data)
check_2 = set.difference(after_data, common_data)
dropped = set.difference(check_1, check_2)
if len(dropped) > 0:
    print("WARNING: " + str(list(dropped)) + " was dropped since either the before or after file doesn't exist")
    
comparison_data = list(common_data)
comparison_data.sort()

In [36]:
RESULT_FILENAME = "Implementation_check_" + str(datetime.now().strftime("%Y-%m-%d_%H-%M-%S")) + ".xlsx"
visual_df = pd.DataFrame()

for db_name in comparison_data:
    # Connection to the database
    con = sqlite3.connect(db_name + '.db')
    cur = con.cursor()
    
    # Data insertion into the database
    insert_data_into_db(db_name, cur)
    
    # Get unique tables from the current database
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    cur.execute(query)
    all_tables = [elem[0] for elem in cur.fetchall()]
    unique_tables = set()
    for table in all_tables:
        suffix_index = table.rfind("_")
        unique_tables.add(table[:suffix_index])
       
    # Check if the both after and before version exists
    unique_tables = list(unique_tables)
    unique_tables.sort()
    for table_name in unique_tables:
        condition1 = (table_name + "_" + BEFORE_IMPLMENT_KEYWORD) not in all_tables
        condition2 = (table_name + "_" + AFTER_IMPLMENT_KEYWORD) not in all_tables
        if(condition1):
            print("WARNING: The before information / table does not exist for table:", table_name)
            unique_tables.discard(table_name)
            
        if(condition2):
            print("WARNING: The after information / table does not exist for table:", table_name)
            unique_tables.discard(table_name)
    
    if not os.path.isfile(RESULT_FILENAME):
        wb = openpyxl.Workbook()
        wb.save(RESULT_FILENAME)
    wb = load_workbook(filename = RESULT_FILENAME)
    ws = wb.create_sheet(title = db_name)
    wb.save(RESULT_FILENAME)
    
    # Posting the results to excel and create graphs
    for table_name in unique_tables:
        result_df = create_result_df(con, table_name)
        write_to_excel_with_formatting(result_df, db_name)

    con.close()
    os.remove(db_name + '.db')

wb = load_workbook(RESULT_FILENAME)
del wb["Sheet"]
wb.save(RESULT_FILENAME)
